<a href="https://colab.research.google.com/github/Amanrathi087/NLP-Project-2-E-commerce-Product-Scraper/blob/main/NLP_Project_2_E_commerce_Product_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ============================================================
# 🛒 Project 2: E-Commerce Product Scraper (Flipkart)
# ============================================================

!pip install requests beautifulsoup4 pandas lxml openpyxl

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1 – choose a search keyword
search = "laptop"   # 🔍 change to "mobile", "headphones", etc.
url = f"https://www.flipkart.com/search?q={search}"

# Step 2 – fetch the page
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# Step 3 – extract product info
products = []

# 🔹 this loop defines 'item', so inside it we can use item.find(...)
for item in soup.find_all("div", {"class": "_1AtVbE"})[:50]:
    # name can appear under several class names
    name = None
    for cls in ["_4rR01T", "IRpwTa", "s1Q9rs"]:
        tag = item.find("div", {"class": cls}) or item.find("a", {"class": cls})
        if tag:
            name = tag.text.strip()
            break

    # price
    price_tag = item.find("div", {"class": "_30jeq3"})
    price = price_tag.text.strip("₹").replace(",", "") if price_tag else None

    # rating
    rating_tag = item.find("div", {"class": "_3LWZlK"})
    rating = rating_tag.text if rating_tag else None

    if name and price:
        products.append({
            "Name": name,
            "Price": price,
            "Rating": rating if rating else "N/A"
        })

# Step 4 – make a DataFrame
df = pd.DataFrame(products)
if df.empty:
    raise ValueError("❌ No products found! Flipkart page structure may have changed.")

df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"].replace("N/A", None), errors="coerce")

print(f"✅ Scraped {len(df)} products successfully!")
display(df.head())

# Step 5 – analysis
avg_price = df["Price"].mean()
top_product = df.loc[df["Rating"].idxmax()]

print("\n💰 Average Price: ₹", round(avg_price, 2))
print("\n⭐ Top-Rated Product:")
display(top_product.to_frame().T)

# Step 6 – save to Excel
df.to_excel("flipkart_products.xlsx", index=False)
print("\n📁 Saved results as flipkart_products.xlsx")


ValueError: ❌ No products found! Flipkart page structure may have changed.